In [2]:
import cv2
import numpy as np
from PIL import Image
from ultralytics import YOLO

In [4]:
model_path = "./models/yolov8n.pt"
model = YOLO(model_path)

In [15]:
img_path_1 = "./photos/apprentices.jpg"
img_path_2 = "./photos/digital_field_worker.jpg"
img_path_3 = "./photos/JS in sunset.jpg"

In [5]:
model.info()

YOLOv8n summary: 225 layers, 3,157,200 parameters, 0 gradients, 8.9 GFLOPs


(225, 3157200, 0, 8.8575488)

In [23]:
# Predict
results = model(img_path_1)
print(results)


image 1/1 /Users/TSUNDV/equinor/inspection-data-analyzer/anonymizer/photos/apprentices.jpg: 448x640 2 persons, 46.7ms
Speed: 1.5ms preprocess, 46.7ms inference, 0.3ms postprocess per image at shape (1, 3, 448, 640)
Results saved to /Users/TSUNDV/equinor/inspection-data-analyzer/runs/detect/predict
2 labels saved to /Users/TSUNDV/equinor/inspection-data-analyzer/runs/detect/predict/labels
[ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ba

In [38]:
# Predict with segmentation
model_seg = YOLO("./models/yolov8n-seg.pt")

In [ ]:
image_path = "./photos/JS in sunset.jpg"
im1 = cv2.imread(image_path)
results = model_seg.predict(source=im1, save=True, save_txt=True)

In [49]:
im1 = cv2.imread(img_path_2)
results = model_seg.predict(source=im1, classes=[0], save=True) #, stream=True)


0: 448x640 1 person, 47.7ms
Speed: 1.6ms preprocess, 47.7ms inference, 0.9ms postprocess per image at shape (1, 3, 448, 640)
Results saved to /Users/TSUNDV/equinor/inspection-data-analyzer/runs/segment/predict4
1 label saved to /Users/TSUNDV/equinor/inspection-data-analyzer/runs/segment/predict4/labels


In [79]:
raw_image = np.array(Image.open(img_path_3))
CLASS_PERSON = 0

results = model_seg.predict(source=raw_image, classes=[CLASS_PERSON], stream=True)

image_anonymized = raw_image

for result in results:
    if not result.masks:
        print("No masks found")
        break

    masks = result.masks.data.cpu().numpy()
    for mask in masks:
        h, w, _ = raw_image.shape
        mask = cv2.resize(mask, (w, h))

        colored_mask = np.expand_dims(mask, 0).repeat(3, axis=0)
        colored_mask = np.moveaxis(colored_mask, 0, -1)

        color = (0, 0, 0)
        masked = np.ma.MaskedArray(image_anonymized, mask=colored_mask, fill_value=color)
        image_anonymized = masked.filled()

im = Image.fromarray(image_anonymized)
im.save("your_file3.jpeg")



0: 448x640 (no detections), 43.6ms
No masks found
